1 - Convertendo os arquivos csv para parquet

In [0]:
### Importando as bibliotecas
import glob

In [0]:
### Declarando as variáveis
diretorio_csv = "/mnt/work/EXPERIMENTACAO/ATHENA/Teste/"
diretorio_parquet = "/mnt/work/EXPERIMENTACAO/ATHENA/Teste/"
qtd_caracteres_dir = len("/dbfs" + diretorio_csv)

In [0]:
### Cria uma lista com os arquivos csv
lista_arquivos = sorted(glob.glob("/dbfs" + diretorio_csv + "*.csv"))
print(lista_arquivos)

['/dbfs/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_customers_dataset.csv', '/dbfs/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_geolocation_dataset.csv', '/dbfs/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_order_items_dataset.csv', '/dbfs/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_order_payments_dataset.csv', '/dbfs/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_order_reviews_dataset.csv', '/dbfs/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_orders_dataset.csv', '/dbfs/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_products_dataset.csv', '/dbfs/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_sellers_dataset.csv', '/dbfs/mnt/work/EXPERIMENTACAO/ATHENA/Teste/product_category_name_translation.csv']

In [0]:
### Monta cursor para converter cada arquivo pra parquet
for x in lista_arquivos:
    df = (spark.read      
    .option("header", "true")
    .option("inferSchema", "true")
    .option("sep", ",")  
    .csv(x[5:]))                            ### Pega o caminho a partir de /dbfs
    x = x[:-4]                              ### Tira a extensão do arquivo
    df.write.parquet(diretorio_parquet + x[qtd_caracteres_dir:] + ".parquet") 

/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_customers_dataset.csv
/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_geolocation_dataset.csv
/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_order_items_dataset.csv
/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_order_payments_dataset.csv
/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_order_reviews_dataset.csv
/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_orders_dataset.csv
/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_products_dataset.csv
/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_sellers_dataset.csv
/mnt/work/EXPERIMENTACAO/ATHENA/Teste/product_category_name_translation.csv

2 - Separando o df com a latitude e a longitude e indo no geopy e viacep

In [0]:
### Instala o geopy e o pycep se necessário
%pip install geopy
%pip install pycep-correios

Python interpreter will be restarted.
Collecting pycep-correios
 Downloading pycep_correios-5.0.0-py2.py3-none-any.whl (7.1 kB)
Collecting zeep>=2.0.0
 Downloading zeep-4.1.0-py2.py3-none-any.whl (100 kB)
Requirement already satisfied: requests>=2.7.0 in /databricks/python3/lib/python3.8/site-packages (from pycep-correios) (2.25.1)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.7.0->pycep-correios) (2020.12.5)
Requirement already satisfied: chardet<5,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.7.0->pycep-correios) (4.0.0)
Requirement already satisfied: urllib3<1.27,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.7.0->pycep-correios) (1.25.11)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.7.0->pycep-correios) (2.10)
Requirement already satisfied: pytz in /databricks/python3/lib/python3.8/site-packages (from zeep>=2.0.0->pycep-correios) (2020.5)
Requirement already satisfied: attrs>=17.2.0 in /databricks/python3/lib/python3.8/site-packages (from zeep>=2.0.0->pycep-correios) (20.3.0)
Collecting requests-file>=1.5.1
 Downloading requests_file-1.5.1-py2.py3-none-any.whl (3.7 kB)
Requirement already satisfied: isodate>=0.5.4 in /databricks/python3/lib/python3.8/site-packages (from zeep>=2.0.0->pycep-correios) (0.6.0)
Collecting platformdirs>=1.4.0
 Downloading platformdirs-2.5.1-py3-none-any.whl (14 kB)
Collecting lxml>=4.6.0
 Downloading lxml-4.8.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (6.9 MB)
Collecting cached-property>=1.3.0
 Downloading cached_property-1.5.2-py2.py3-none-any.whl (7.6 kB)
Collecting requests-toolbelt>=0.7.1
 Downloading requests_toolbelt-0.9.1-py2.py3-none-any.whl (54 kB)
Requirement already satisfied: six in /databricks/python3/lib/python3.8/site-packages (from isodate>=0.5.4->zeep>=2.0.0->pycep-correios) (1.15.0)
Installing collected packages: requests-toolbelt, requests-file, platformdirs, lxml, cached-property, zeep, pycep-correios
Successfully installed cached-property-1.5.2 lxml-4.8.0 platformdirs-2.5.1 pycep-correios-5.0.0 requests-file-1.5.1 requests-toolbelt-0.9.1 zeep-4.1.0
Python interpreter will be restarted.

In [0]:
### Importa as bibliotecas
from geopy.geocoders import Nominatim
from pycep_correios import get_address_from_cep, WebService

In [0]:
### Verifica se os arquivos parquet tão lá
print (sorted(glob.glob("/dbfs" + diretorio_parquet + "*.parquet")))

['/dbfs/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_customers_dataset.parquet', '/dbfs/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_geolocation_dataset.parquet', '/dbfs/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_order_items_dataset.parquet', '/dbfs/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_order_payments_dataset.parquet', '/dbfs/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_order_reviews_dataset.parquet', '/dbfs/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_orders_dataset.parquet', '/dbfs/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_products_dataset.parquet', '/dbfs/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_sellers_dataset.parquet', '/dbfs/mnt/work/EXPERIMENTACAO/ATHENA/Teste/product_category_name_translation.parquet']

In [0]:
### Separo o arquivo de geolocation
file_name = ["/mnt/work/EXPERIMENTACAO/ATHENA/Teste/olist_geolocation_dataset.parquet"]

In [0]:
### Monto o df com o arquivo de geolocation
df = spark.read.parquet(*file_name)

In [0]:
### Confirmo que ta certinho
df.show(10)

+---------------------------+-------------------+-------------------+----------------+-----------------+
geolocation_zip_code_prefix| geolocation_lat| geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+-------------------+-------------------+----------------+-----------------+
 64078| -5.113478401970498|-42.752681112783364| teresina| PI|
 64056| -5.067542694690987| -42.76521910068536| teresina| PI|
 64007| -5.058476960631243| -42.81858896344071| teresina| PI|
 64012| -4.993013285763618| -42.84385975371684| teresina| PI|
 64000| -5.08119555080314| -42.80046403888144| teresina| PI|
 64012|-4.9920123583732705|-42.841503368137424| teresina| PI|
 64002| -5.064942461411257|-42.810477615387235| teresina| PI|
 64056| -5.065044354921431| -42.76757323281915| teresina| PI|
 64015| -5.102135063120522|-42.784766870596464| teresina| PI|
 64001| -5.085806876475155| -42.7940314421668| teresina| PI|
+---------------------------+-------------------+-------------------+----------------+-----------------+
only showing top 10 rows

In [0]:
# Pego a latitude e a longitude do df e com isso busco o cep e o endereço
dataCollect = df.collect()
for row in dataCollect:
    v_lat_lon = (str(row['geolocation_lat']) + ", " + str(row['geolocation_lng']))
    print(v_lat_lon)
    
    geolocator = Nominatim(user_agent="MBA_FIAP")
    location = geolocator.reverse(v_lat_lon)
    cep = (location.raw['address']['postcode'])
    
    if len(cep) < 8:
        cep = cep.zfill(8)
        cep = '{}-{}'.format(cep[:5], cep[5:8])
    print(cep) 
    
    address = get_address_from_cep(cep, webservice=WebService.VIACEP)
    
    print(address['logradouro'])
    
    print('-----------')

###### É preciso ver o que fazer quando o o retorno da geopy não for CEP

-5.113478401970498, -42.752681112783364
64076-340
Rua Professor Cláudio Ferreira
-----------
-5.067542694690987, -42.76521910068536
64055-330
Rua Mundinho Ferraz
-----------
-5.058476960631243, -42.81858896344071
64007500
Rua Lucrécio Avelino
-----------
-4.993013285763618, -42.84385975371684
64012355
Rua Maria Mirto de Sá
-----------
-5.08119555080314, -42.80046403888144
64000-512
Avenida Miguel Rosa
-----------
-4.9920123583732705, -42.841503368137424
-5.0004536 -42.8637018

--------------------------------------------------------------------------- 
 InvalidCEP Traceback (most recent call last)
 <command-2138753537998818> in <module> 
 14 print ( cep ) 
 15 
 ---> 16 address = get_address_from_cep ( cep , webservice = WebService . VIACEP ) 
 17 
 18 print ( address [ 'logradouro' ] ) 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-5b3c09cf-6968-4c8c-b2dd-603b226698c6/lib/python3.8/site-packages/pycep_correios/client.py in get_address_from_cep (cep, webservice) 
 35 
 36 if webservice == WebService . VIACEP : 
 ---> 37 return viacep . fetch_address ( cep ) 
 38 
 39 if webservice == WebService . APICEP : 

 /local_disk0/.ephemeral_nfs/envs/pythonEnv-5b3c09cf-6968-4c8c-b2dd-603b226698c6/lib/python3.8/site-packages/pycep_correios/viacep.py in fetch_address (cep) 
 31 
 32 elif response . status_code == 400 : 
 ---> 33 raise exceptions . InvalidCEP ( ) 
 34 
 35 else : 

 InvalidCEP :

In [0]:
#################### Blodo de teste
location = geolocator.reverse('-5.0004536, -42.8637018')
print(location.raw)

{'place_id': 74489739, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 6974678606, 'lat': '-5.000499', 'lon': '-42.8636446', 'display_name': 'EMEF - João Azevedo, Rua Tenente Araújo, Santa Maria, Timon, Região Geográfica Imediata de Teresina, Região Integrada de Desenvolvimento da Grande Teresina, Região Geográfica Intermediária de Teresina, Piauí, Região Nordeste, -5.0004536 -42.8637018, Brasil', 'address': {'amenity': 'EMEF - João Azevedo', 'road': 'Rua Tenente Araújo', 'suburb': 'Santa Maria', 'city': 'Timon', 'municipality': 'Região Geográfica Imediata de Teresina', 'county': 'Região Integrada de Desenvolvimento da Grande Teresina', 'state_district': 'Região Geográfica Intermediária de Teresina', 'state': 'Piauí', 'region': 'Região Nordeste', 'postcode': '-5.0004536 -42.8637018', 'country': 'Brasil', 'country_code': 'br'}, 'boundingbox': ['-5.000549', '-5.000449', '-42.8636946', '-42.8635946']}